# Computing Baseline and Standard Integral
## Importing Libraries

In [1]:
# Base libraries
import math
import numpy as np
import scipy.integrate as integrate
from tqdm import tqdm
from scipy.special import erf
import pickle
import itertools

from SALib.sample import saltelli
from SALib.analyze import sobol

# Personal libraries
import sixtrackwrap as sx

from parameters import *

import time

## Setup coordinates

In [2]:
DA_b = {}
error_b = {}

alpha_preliminary_values = np.linspace(-1.0, 1.0, baseline_samples)
alpha_values = np.arccos(alpha_preliminary_values) / 2
theta1_values = np.linspace(0.0, np.pi * 2.0, baseline_samples, endpoint=False)
theta2_values = np.linspace(0.0, np.pi * 2.0, baseline_samples, endpoint=False)

d_preliminar_alpha = alpha_preliminary_values[1] - alpha_preliminary_values[0]
d_theta1 = theta1_values[1] - theta1_values[0]
d_theta2 = theta2_values[1] - theta2_values[0]

alpha_mesh, theta1_mesh, theta2_mesh = np.meshgrid(alpha_values, theta1_values, theta2_values, indexing='ij')

alpha_flat = alpha_mesh.flatten()
theta1_flat = theta1_mesh.flatten()
theta2_flat = theta2_mesh.flatten()

## Baseline

In [3]:
time_start = time.time()

# Data generation
engine = sx.radial_scanner(alpha_flat, theta1_flat, theta2_flat, d_r, starting_step=starting_step)

engine.scan(max_turns, min_turns, batch_size=batch_size)

radiuses = engine.extract_DA(turn_sampling)
radiuses = radiuses.reshape((baseline_samples, baseline_samples, baseline_samples, len(turn_sampling)))

data_b = radiuses

elapsed_time_engine = time.time() - time_start

Active radiuses: 35937 / 35937
Sample size per active radius: 1
Expected execution time for step: nan
Elapsed time for whole iteration: 27.73
Time per single iteration: 1.543267069049473e-05
r: 15.0 . Turns to do: 50 . Min found: 50
Active radiuses: 35937 / 35937
Sample size per active radius: 1
Expected execution time for step: 27.73
Elapsed time for whole iteration: 27.64
Time per single iteration: 1.538009507152484e-05
r: 20.0 . Turns to do: 50 . Min found: 35
Active radiuses: 35937 / 35937
Sample size per active radius: 1
Expected execution time for step: 27.64
Elapsed time for whole iteration: 27.40
Time per single iteration: 1.524844770555979e-05
r: 25.0 . Turns to do: 50 . Min found: 4
Active radiuses: 33831 / 35937
Sample size per active radius: 1
Expected execution time for step: 25.79
Elapsed time for whole iteration: 29.54
Time per single iteration: 1.7465159174982358e-05
r: 30.0 . Turns to do: 50 . Min found: 0
Active radiuses: 20159 / 35937
Sample size per active radius: 1

In [4]:
time_start = time.time()

radiuses = data_b
# Computing DA
DA = []
error_list = []
mod_radiuses = radiuses.copy()
mod_radiuses = np.power(radiuses, 4)

mod_radiuses1 = integrate.simps(mod_radiuses, x=theta1_values, axis=1)
error_radiuses1 = np.absolute(
    (mod_radiuses1 - integrate.simps(mod_radiuses[:,::2,:], x=theta1_values[::2], axis=1)) / mod_radiuses1
)
error_radiuses1 = np.average(error_radiuses1, axis=1)

mod_radiuses2 = integrate.simps(mod_radiuses1, x=theta2_values, axis=1)
error_radiuses2 = np.absolute(
    (mod_radiuses2 - integrate.simps(mod_radiuses1[:,::2], x=theta2_values[::2], axis=1)) / mod_radiuses2
)
error_radiuses2 += error_radiuses1
error_radiuses2 = np.average(error_radiuses2, axis=0)

mod_radiuses3 = integrate.simps(mod_radiuses2, x=alpha_preliminary_values, axis=0)
error_radiuses3 = np.absolute(
    (mod_radiuses3 - integrate.simps(mod_radiuses2[::2], x=alpha_preliminary_values[::2], axis=0)) / mod_radiuses3
)
error_radiuses3 += error_radiuses2

error_raw = mod_radiuses3/ (2 * theta1_values[-1] * theta2_values[-1]) * error_radiuses3
error = 0.25 * np.power(mod_radiuses3 / (2 * theta1_values[-1] * theta2_values[-1]), -3/4) * error_raw

for i in range(len(turn_sampling)):
    DA.append(
        np.power(
            mod_radiuses3[i] / (2 * theta1_values[-1] * theta2_values[-1]),
            1/4
        )
    )
    error_list.append(error[i])
DA_b = np.asarray(DA)
error_b = np.asarray(error_list)

### Saving Data

In [5]:
with open(savepath + "data/raw_data_b.pkl", 'wb') as f:
    pickle.dump(data_b, f, protocol=4)
    
with open(savepath + "data/DA_b.pkl", 'wb') as f:
    pickle.dump(DA_b, f, protocol=4)
    
with open(savepath + "data/error_b.pkl", 'wb') as f:
    pickle.dump(error_b, f, protocol=4)

## Monte Carlo Comparison

In [6]:
radiuses = data_b.reshape(-1, data_b.shape[-1])
    
average = np.average(np.power(radiuses, 4), axis=0)
error = np.std(np.power(radiuses, 4), axis=0) / np.sqrt(radiuses.shape[0])

DA_b_mc = np.power(average, 1/4)
error_b_mc = 0.25 * np.power(average, -3/4) * error

### Saving Data

In [7]:
with open(savepath + "data/DA_b_mc.pkl", 'wb') as f:
    pickle.dump(DA_b_mc, f, protocol=4)
    
with open(savepath + "data/error_b_mc.pkl", 'wb') as f:
    pickle.dump(error_b_mc, f, protocol=4)

## Standard Integral

In [9]:
DA_1 = {}
error_1 = {}
base_radiuses = data_b

values = [2]
while True:
    if (baseline_samples - 1) // values[-1] > 4:
        values.append(values[-1] * 2)
    else:
        break

for i in values:
    radiuses = base_radiuses[::i, ::i, ::i]
    DA = []
    error_list = []
    mod_radiuses = radiuses.copy()
    mod_radiuses = np.power(radiuses, 4)

    mod_radiuses1 = integrate.simps(mod_radiuses, x=theta1_values[::i], axis=1)
    error_radiuses1 = np.absolute(
        (mod_radiuses1 - integrate.simps(mod_radiuses[:,::2,:], x=theta1_values[::i * 2], axis=1)) / mod_radiuses1
    )
    error_radiuses1 = np.average(error_radiuses1, axis=1)

    mod_radiuses2 = integrate.simps(mod_radiuses1, x=theta2_values[::i], axis=1)
    error_radiuses2 = np.absolute(
        (mod_radiuses2 - integrate.simps(mod_radiuses1[:,::2], x=theta2_values[::i * 2], axis=1)) / mod_radiuses2
    )
    error_radiuses2 += error_radiuses1
    error_radiuses2 = np.average(error_radiuses2, axis=0)

    mod_radiuses3 = integrate.simps(mod_radiuses2, x=alpha_preliminary_values[::i], axis=0)
    error_radiuses3 = np.absolute(
        (mod_radiuses3 - integrate.simps(mod_radiuses2[::2], x=alpha_preliminary_values[::i * 2], axis=0)) / mod_radiuses3
    )
    error_radiuses3 += error_radiuses2

    error_raw = mod_radiuses3/ (2 * theta1_values[-1] * theta2_values[-1]) * error_radiuses3
    error = 0.25 * np.power(mod_radiuses3 / (2 * theta1_values[-1] * theta2_values[-1]), -3/4) * error_raw

    for j in range(len(turn_sampling)):
        DA.append(
            np.power(
                mod_radiuses3[j] / (2 * theta1_values[-1] * theta2_values[-1]),
                1/4
            )
        )
        error_list.append(error[j])
    DA_1[radiuses.shape[0]**3] = np.asarray(DA)
    error_1[radiuses.shape[0]**3] = np.asarray(error_list)

### Saving Data

In [10]:
with open(savepath + "data/DA_1.pkl", 'wb') as f:
    pickle.dump(DA_1, f, protocol=4)
    
with open(savepath + "data/error_1.pkl", 'wb') as f:
    pickle.dump(error_1, f, protocol=4)
    
elapsed_time_processing = time.time() - time_start

with open(savepath + "data/time_1.pkl", "wb") as f:
    pickle.dump((elapsed_time_engine, elapsed_time_processing), f, protocol=4)